In [1]:
import torch
import numpy as np
from tqdm import tqdm
import copy
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset, Dataset, DatasetDict
from torch.nn.utils.rnn import pad_sequence

In [2]:
dataset_org = load_dataset("beratcmn/instruction-turkish-poems")

In [3]:
dataset = dataset_org
dataset.remove_columns(['instruction','id', 'rating'])

DatasetDict({
    train: Dataset({
        features: ['title', 'poet', 'poem'],
        num_rows: 4961
    })
})

In [4]:
poet_data = {}
poets_with_100_poems_data = []
i = 0

for example in dataset['train']:
    poet = example['poet']
    title = example['title']
    poem = example['poem']
    if poet not in poet_data:
        poet_data[poet] = {'poems': [], 'titles': []}
    poet_data[poet]['poems'].append(example['poem'])
    poet_data[poet]['titles'].append(example['title'])
    
    if len(poet_data[poet]['poems']) == 100:
        i += 1
        poets_with_100_poems_data.append({'poet': poet, 'title': title, 'poem': poem})
    if i == 5:
        break
poets_with_100_poems_dataset = Dataset.from_dict({'poet': [item['poet'] for item in poets_with_100_poems_data],
                                                  'title': [item['title'] for item in poets_with_100_poems_data],
                                                  'poem': [item['poem'] for item in poets_with_100_poems_data]})

dataset_dict = DatasetDict({"train": poets_with_100_poems_dataset})

In [5]:
poets_with_100_poems_dataset['poet']

['Abdurrahim Karakoç',
 'Necip Fazıl Kısakürek',
 'Ümit Yaşar Oğuzcan',
 'Pir Sultan Abdal',
 'Can Yücel']

In [6]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['poet', 'title', 'poem'],
        num_rows: 5
    })
})

In [7]:
dataset_dict['train']['poem'][0]

"Çelik testereyle kestim suları\nYıkadım duvara astım suları..\n   Düşümde düşüme girdim dün gece.\n\nBuluta yaslandım ışığı tuttum.\nSeni hatırladım, seni unutdum..\n   Kendimi kendime sordum dün gece.\n\nTopladım yolları eyledim yumak\nMusalladan gayri görmedim durak...\n   Durmadan düşünüp durdum dün gece.\n\nToprağı boyadım otlar ağladı\nOturdum kalkmadım atlar ağladı..\n   Tuttum yorgunluğu yordum dün gece.\n\nDertler gecikince gidip yokladım\nYırtık bohçalarda umut sakladım..\n   Kırgınlık bağını kırdım dün gece.\n\nŞişelerde mahkûm çiçek kokusu\nYağdı yüreğime renk renk korkusu..\n   Yok yere yokluğu vurdum dün gece.\n\nAy doğdu, gölgeler çöktü üstüme\nHicran alev alev aktı üstüme..\n   Gözümü yollarda gördüm dün gece.\n\nAydınlığa koştum karanlık çıktı\nHer sevgi, her vefa bir anlık çıktı..\n   Güç-belâ ben bana vardım dün gece.\n\nDosta şiir yazdım 'hatıra' dedim\nBelki bir dost gele otura dedim..\n   Gönlümü toprağa serdim dün gece\n\n(Beşinci Mevsim)"

In [8]:
class PoemDataset(Dataset):
    def __init__(self, dataset_dict, split, tokenizer, max_length=512):
        """
        dataset: A list of dictionaries with keys 'poet', 'title', and 'poem'.
        tokenizer: Tokenizer to be used for encoding text.
        max_input_length: Maximum length of the input sequence.
        max_target_length: Maximum length of the target sequence.
        """
        self.dataset = dataset_dict[split]
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        data = self.dataset[idx]
        poet, title, poem = data['poet'], data['title'], data['poem']

        # Preparing the input text and target text
        input_text = f"[POET]{poet} [TITLE]{title}"
        target_text = poem

        # Encoding the input and target text
        input_encoding = self.tokenizer(
            input_text,
            padding='max_length',
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_length,
            return_attention_mask=True,
            return_tensors='pt',
        )

        target_encoding = self.tokenizer(
            target_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        input_ids = input_encoding['input_ids'].squeeze(0)
        attention_mask = input_encoding['attention_mask'].squeeze(0)
        target_ids = target_encoding['input_ids'].squeeze(0)

        #input_ids[input_ids == self.tokenizer.pad_token_id] = -100
        target_ids[target_ids == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': target_ids
        }

In [18]:
model_name = "sberbank-ai/mGPT"
model = GPT2LMHeadModel.from_pretrained(model_name, gradient_checkpointing=True, use_cache=False)
model.resize_token_embeddings(len(tokenizer))

Embedding(100002, 2048)

In [11]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
special_tokens_dict = {'additional_special_tokens': ['[POET]', '[TITLE]']}
tokenizer.add_special_tokens(special_tokens_dict)

2

In [53]:
train_dataset = PoemDataset(dataset_dict, 'train', tokenizer)
dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [29]:
model_org = copy.copy(model)

In [47]:
model = copy.copy(model_org)

In [49]:
# Freeze transformer layers except the first and the last one. Do not freeze any layernorms
for n, p in model.named_parameters():
    if 'transformer.h' in n:
        layer_num = int(n.split('.')[2])
        if 'ln_' not in n and layer_num > 0 and layer_num < 23:
            p.requires_grad = False
            #print('Freeze', n)

In [50]:
optimizer = torch.optim.SGD(params=model.parameters(), lr=5e-5)

In [54]:
model.train()
for epoch in range(3):
    print(f'Epoch {epoch}')
    progressbar = tqdm(dataloader)
    losses = []

    for batch in progressbar:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        labels = batch['labels'].to(model.device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        if loss is not None:
            loss.backward()
            optimizer.step()

            losses.append(loss.item())
            progressbar.set_description("Loss: %.3f" % np.mean(losses[-10:]))

Epoch 0


Loss: 9.328: 100%|███████████████████████████████████████████████████████████████████████| 5/5 [01:53<00:00, 22.69s/it]


Epoch 1


Loss: 9.193: 100%|███████████████████████████████████████████████████████████████████████| 5/5 [01:53<00:00, 22.71s/it]


Epoch 2


Loss: 9.068: 100%|███████████████████████████████████████████████████████████████████████| 5/5 [01:54<00:00, 22.99s/it]


In [22]:
torch.save(model.state_dict(), 'model.pth')

In [12]:
model_name = "sberbank-ai/mGPT"
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [15]:
input_ids = tokenizer.encode("[POET]Abdurrahim Karakoç [TITLE]Huzursuzluk", return_tensors = "pt").cuda()
input_ids = input_ids.to(model.device)
out = model.generate(
        input_ids,
        min_length = 100,
        max_length = 100,
        eos_token_id = 5,
        do_sample = True,
        top_k = 4,
        top_p = 0.88,
        no_repeat_ngram_size=3)
generated_text = list(map(tokenizer.decode, out))[0]
print('---')
print(generated_text)


---
[POET] Abdurrahim Karakoç  [TITLE] Huzursuzluk, sıkıntı ve endişe ile başa çıkmada en etkili yöntemlerden biri de yatıştırıcı bir dil kullanmaktır. Yatıştırıcılık, bir kişinin kendine olan güvenini artırarak kişiyi rahatlatabilir. Yapılan araştırmalar, yatışma ile baş etmenin, stresi ve endiseli kişilerin daha az


In [ ]:
input_ids

tensor([[     0, 100000,     52,    424,    607,  51378,   4395,  98196,   4023,
            455,  16767,    227, 100001,     63,    327,   6417,    307,   1072,
            361,      5]], device='cuda:0')

In [ ]:
input_idss = tokenizer.encode(generated_text, return_tensors = "pt").cuda()

In [ ]:
input_idss

tensor([[     0,    227, 100000,  70342,    607,  51378,   4395,  98196,   4023,
            455,  16767,    227,    227, 100001,   1065,    327,   6417,    307,
           1072,    361,      5,    205,      1,      1,      1,  12269,      1,
              1,    308,    122,      1,      1,   2348,    808,    103,    178,
            130,    128,      1,      1,  25033,      1,      1,  25495,      1,
              1,  86400,      1,      1,  69213,      1,      1,  29431,      1,
              1,    205,      1,  21954,      1,      1,  98004,      1,      1,
           6753,      1,      1,  12834,      1,      1,  74392,      1,      1,
          40664,      1,      1,  42783,      1,      1,  21954,  67609,      1,
              1,  10656,      1,      1,  67609,      1,  10656,  67609,      1,
          86400,  67609,      1,  25495,  14200,      1,      1,  21976,      1,
              1,  33254,      1,      1,  49355,      1,      1,  96134,      1,
              1,  30574,    